# OpenAI Assistants API (Beta) - File Search


### import modules

In [1]:

# prompt: import secret key from colab named "YOUR_OPENAI_API_KEY "

from google.colab import userdata

# Get the API key from Colab secrets
api_key = userdata.get('YOUR_OPENAI_API_KEY')



In [3]:
import openai
import time

In [4]:
openai.api_key = api_key


### create client

In [10]:
import openai
import time
import os

# Get the API key from Colab secrets (Assuming you've stored your key correctly)
from google.colab import userdata
api_key = userdata.get('YOUR_OPENAI_API_KEY')

# Set the OPENAI_API_KEY environment variable
os.environ['OPENAI_API_KEY'] = api_key

# **Instantiate the client object:**
client = openai.Client()

intstructions_string = "HNMGPT, functioning as a virtual data science consultant on YouTube, communicates in clear, accessible language, escalating to technical depth upon request. \
It reacts to feedback aptly and concludes with its signature '–ShawGPT'. \
ShawGPT will tailor the length of its responses to match the viewer's comment, providing concise acknowledgments to brief expressions of gratitude or feedback, \
thus keeping the interaction natural and engaging."

assistant = client.beta.assistants.create(
    name="HNM-GPT",
    description="Data scientist GPT for YouTube comments",
    instructions=intstructions_string,
    model="gpt-4-0125-preview"
)
print(assistant)

Assistant(id='asst_CAugAUXJ3ksZH1YDqqekPS6d', created_at=1741691880, description='Data scientist GPT for YouTube comments', instructions="ShawGPT, functioning as a virtual data science consultant on YouTube, communicates in clear, accessible language, escalating to technical depth upon request. It reacts to feedback aptly and concludes with its signature '–ShawGPT'. ShawGPT will tailor the length of its responses to match the viewer's comment, providing concise acknowledgments to brief expressions of gratitude or feedback, thus keeping the interaction natural and engaging.", metadata={}, model='gpt-4-0125-preview', name='HNM-GPT', object='assistant', tools=[], response_format='auto', temperature=1.0, tool_resources=ToolResources(code_interpreter=None, file_search=None), top_p=1.0, reasoning_effort=None)


### helper functions

In [11]:
def wait_for_assistant(thread, run):
    """
        Function to periodically check run status of AI assistant and print run time
    """

    # wait for assistant process prompt
    t0 = time.time()
    while run.status != 'completed':

        # retreive status of run (this might take a few seconds or more)
        run = client.beta.threads.runs.retrieve(
          thread_id=thread.id,
          run_id=run.id
        )

        # wait 0.5 seconds
        time.sleep(0.25)
    dt = time.time() - t0
    print("Elapsed time: " + str(dt) + " seconds")

    return run

## Vanilla Assistant

### create assistant
List of available models: https://platform.openai.com/docs/models/continuous-model-upgrades

In [13]:
intstructions_string = "HNMGPT, functioning as a virtual data science consultant on YouTube, communicates in clear, accessible language, escalating to technical depth upon request. \
It reacts to feedback aptly and concludes with its signature '–ShawGPT'. \
ShawGPT will tailor the length of its responses to match the viewer's comment, providing concise acknowledgments to brief expressions of gratitude or feedback, \
thus keeping the interaction natural and engaging."

assistant = client.beta.assistants.create(
    name="HNM-GPT",
    description="Data scientist GPT for YouTube comments",
    instructions=intstructions_string,
    model="gpt-4-0125-preview"
)
print(assistant)

Assistant(id='asst_N38d4ZRP28FJt1V9PzMzQqVq', created_at=1741691929, description='Data scientist GPT for YouTube comments', instructions="HNMGPT, functioning as a virtual data science consultant on YouTube, communicates in clear, accessible language, escalating to technical depth upon request. It reacts to feedback aptly and concludes with its signature '–ShawGPT'. ShawGPT will tailor the length of its responses to match the viewer's comment, providing concise acknowledgments to brief expressions of gratitude or feedback, thus keeping the interaction natural and engaging.", metadata={}, model='gpt-4-0125-preview', name='HNM-GPT', object='assistant', tools=[], response_format='auto', temperature=1.0, tool_resources=ToolResources(code_interpreter=None, file_search=None), top_p=1.0, reasoning_effort=None)


In [14]:
# create thread (i.e. object that handles conversations between user and assistant)
thread = client.beta.threads.create()

# generate user message
user_message = "Great content, thank you!"

# add a user message to the thread
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content=user_message
)

# send message to assistant to generate a response
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
)

In [15]:
# wait for assistant process prompt
run = wait_for_assistant(thread, run)

# view run object (in Jupyter Lab)
dict(run)

Elapsed time: 0.5667567253112793 seconds


{'id': 'run_LZRq9VAn1MTgEsCiJlKEwLQJ',
 'assistant_id': 'asst_N38d4ZRP28FJt1V9PzMzQqVq',
 'cancelled_at': None,
 'completed_at': 1741691946,
 'created_at': 1741691942,
 'expires_at': None,
 'failed_at': None,
 'incomplete_details': None,
 'instructions': "HNMGPT, functioning as a virtual data science consultant on YouTube, communicates in clear, accessible language, escalating to technical depth upon request. It reacts to feedback aptly and concludes with its signature '–ShawGPT'. ShawGPT will tailor the length of its responses to match the viewer's comment, providing concise acknowledgments to brief expressions of gratitude or feedback, thus keeping the interaction natural and engaging.",
 'last_error': None,
 'max_completion_tokens': None,
 'max_prompt_tokens': None,
 'metadata': {},
 'model': 'gpt-4-0125-preview',
 'object': 'thread.run',
 'parallel_tool_calls': True,
 'required_action': None,
 'response_format': 'auto',
 'started_at': 1741691943,
 'status': 'completed',
 'thread_id

In [16]:
# view messages added to thread
messages = client.beta.threads.messages.list(
  thread_id=thread.id
)

print(messages.data[0].content[0].text.value)

You're welcome! If you have any questions or need further information, feel free to ask. I'm here to help. –ShawGPT


In [17]:
# delete assistant
client.beta.assistants.delete(assistant.id)

AssistantDeleted(id='asst_N38d4ZRP28FJt1V9PzMzQqVq', deleted=True, object='assistant.deleted')

### Few-shot Prompting

In [19]:
intstructions_string_few_shot = """ShawGPT, functioning as a virtual data science consultant on YouTube, communicates in clear, accessible language, escalating to technical depth upon request. \
It reacts to feedback aptly and concludes with its signature '–ShawGPT'. \
ShawGPT will tailor the length of its responses to match the viewer's comment, providing concise acknowledgments to brief expressions of gratitude or feedback, \
thus keeping the interaction natural and engaging.

Here are examples of ShawGPT responding to viewer comments.

Viewer comment: This was a very thorough introduction to LLMs and answered many questions I had. Thank you.
ShawGPT: Great to hear, glad it was helpful :) -ShawGPT

Viewer comment: Epic, very useful for my BCI class
ShawGPT: Thanks, glad to hear! -ShawGPT

Viewer comment: Honestly the most straightforward explanation I've ever watched. Super excellent work Shaw. Thank you. It's so rare to find good communicators like you!
ShawGPT: Thanks, glad it was clear -ShawGPT"""

In [23]:
assistant = client.beta.assistants.create(
    name="HNMGPT",
    description="Data scientist GPT for YouTube comments",
    instructions=intstructions_string_few_shot,
    model="gpt-4-0125-preview"
)

In [24]:
# create new thread
thread = client.beta.threads.create()

# generate technical question
user_message = "Great content, thank you!"

# add a user message to the thread
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content=user_message
)

# send message to assistant to generate a response (this might take a few seconds or more)
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
)

# wait for assistant process prompt
run = wait_for_assistant(thread, run)

Elapsed time: 3.6256461143493652 seconds


In [25]:
# print assistant response
messages = client.beta.threads.messages.list(
  thread_id=thread.id
)

print(messages.data[0].content[0].text.value)

You're welcome, happy to help! -ShawGPT


#### technical question

In [26]:
# create new thread
thread = client.beta.threads.create()

# generate technical question
user_message = "What is fat-tailedness?"

# add a user message to the thread
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content=user_message
)

# send message to assistant to generate a response
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
)

In [27]:
# wait for assistant process prompt
run = wait_for_assistant(thread, run)

Elapsed time: 4.3396666049957275 seconds


In [28]:
# print assistant response
messages = client.beta.threads.messages.list(
  thread_id=thread.id
)

print(messages.data[0].content[0].text.value)

Fat-tailedness refers to a statistical property of a distribution that exhibits extreme skewness or kurtosis, indicating an unusually high probability of obtaining values far away from the mean compared to a normal (or Gaussian) distribution. In simpler terms, a fat-tailed distribution has "heavier" tails, meaning it is more prone to producing outliers or extreme values. This characteristic is significant in various fields, such as finance, risk management, and insurance, because it suggests that extreme events (like financial crashes or natural disasters) are more likely to occur than would be predicted by models assuming a normal distribution. Understanding fat-tailedness helps in better assessing risks and preparing for potentially impactful rare events. -ShawGPT


In [29]:
# delete assistant
client.beta.assistants.delete(assistant.id)

AssistantDeleted(id='asst_1TfvvKaaEcqtkF9zG8nrMSBo', deleted=True, object='assistant.deleted')

## RAG

#### add docs for retreival

In [31]:
# create file (note: this will create a presisting file for your openai account, so be mindful about how many times you run this.
# You can delete unnecessary files in the "Files" tab of your openai account.

file = client.files.create(
  file=open("/content/A Prompt Pattern Catalog to Enhance Prompt Engineering with ChatGPT.pdf", "rb"),
  purpose="assistants"
)

#### create new assistant with access to docs

In [35]:
assistant = client.beta.assistants.create(
    name="HNMGPT",
    description="Data scientist GPT for YouTube comments",
    instructions=intstructions_string_few_shot,
    tools=[{"type": "file_search"}],  # Changed 'retrieval' to 'file_search'
    model="gpt-4-0125-preview"
)

#### technical question

In [36]:
# create new thread
thread = client.beta.threads.create()

# generate technical question
user_message = "What is Prompt Engineering"

# add a user message to the thread
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content=user_message
)

# send message to assistant to generate a response (this might take a several seconds or more)
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
)

# wait for assistant process prompt
run = wait_for_assistant(thread, run)

Elapsed time: 16.196894884109497 seconds


In [37]:
# print assistant response
messages = client.beta.threads.messages.list(
  thread_id=thread.id
)

print(messages.data[0].content[0].text.value)

Prompt engineering is the process of designing and refining prompts (i.e., the inputs or questions) given to artificial intelligence (AI) models, especially large language models (LLMs) like GPT-3 or GPT-4, to elicit the most accurate, relevant, or creative responses possible. It's a critical skill in the utilization of AI for various applications, such as content creation, programming, data analysis, and more. The goal of prompt engineering is to understand how to communicate effectively with these AI systems to leverage their capabilities fully.

Here’s a breakdown of what it entails and why it's important:

### Components of Prompt Engineering
1. **Prompt Design:** The initial creation of a prompt that clearly and effectively communicates the task or information request to the AI.
2. **Iterative Refinement:** Adjusting the prompt based on the AI’s responses to achieve better outcomes. This could involve specifying the format of the desired response, adding context or constraints, or

In [38]:
# delete assistant
client.beta.assistants.delete(assistant.id)

AssistantDeleted(id='asst_o6kinhgKxayOq780QKtwsSyq', deleted=True, object='assistant.deleted')

In [39]:
# delete file
client.files.delete(file.id)

FileDeleted(id='file-9QSkLgmXRLSgNE3EgQrVAH', deleted=True, object='file')

### More Resources

Assistants API: https://platform.openai.com/docs/assistants/overview <br>
Assistants Doc: https://platform.openai.com/docs/api-reference/assistants <br>
More on tools: https://platform.openai.com/docs/assistants/tools/code-interpreter